    PUMP IT UP

Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, which need some repairs, and which don't work at all? This is an intermediate-level practice competition. Predict one of these three classes based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

An interactive course exploring this dataset is currently offered by DataCamp.com!

Competition End Date: Jan. 28, 2017, 11:59 p.m.

This competition is for learning and exploring, so the deadline may be extended in the future.


* [Git Hub Repo](https://github.com/msampathkumar/datadriven_pumpit)
* [Git Hub Report](https://github.com/msampathkumar/datadriven_pumpit/blob/master/capstone_proposal.mdown)
* [Features Details](https://www.drivendata.org/competitions/7/page/25/)


TODO:

1. Variance Threshold Dict
2. know variance threshold for removed columns in 


# Import Libraries

In [ ]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

np.random.seed(69572)

%matplotlib inline

%load_ext writeandexecute

# plt.figure(figsize=(120,10))

small = (4,3)
mid = (10, 8)
large = (12, 8)

# Custom Hacks

### MarkUP Fns

In [ ]:
from __future__ import absolute_import
from IPython.core.getipython import get_ipython
from IPython.core.magic import (Magics, magics_class,  cell_magic)
import sys
from StringIO import StringIO
from markdown import markdown
from IPython.core.display import HTML
 
@magics_class
class MarkdownMagics(Magics):
 
    @cell_magic
    def asmarkdown(self, line, cell):
        buffer = StringIO()
        stdout = sys.stdout
        sys.stdout = buffer
        try:
            exec(cell, locals(), self.shell.user_ns)
        except:
            sys.stdout = stdout
            raise
        sys.stdout = stdout
        return HTML("<p>{}</p>".format(markdown(buffer.getvalue(), extensions=['markdown.extensions.extra'])))
        return buffer.getvalue() + 'test'
 
get_ipython().register_magics(MarkdownMagics)

### Value Counts

In [ ]:
def raw_markup_value_counts(dataframe, max_print_value_counts=30, show_plots=False, figsize=(9, 3)):
    '''
    prints value counts of each feature in data frame
    '''
    if not figsize:
        figsize=(9, 3)
    mydf = pd.DataFrame.copy(dataframe)
    i = 0
    raw_markup_data = []
    pp = raw_markup_data.append
    pp('''|Col ID|Col Name|UniqCount|Col Values|UniqValCount|''')
    pp('''|------|--------|---------|----------|------------|''')
    for col in mydf.dtypes.index:
        i += 1
        sam = mydf[col]
        sam_value_counts = sam.value_counts()
        tmp = len(sam_value_counts)
        sam_value_counts_len = len(sam_value_counts)
        if 1 < sam_value_counts_len < max_print_value_counts:
            flag = True
            for key, val in dict(sam_value_counts).iteritems():
                if flag:
                    pp('|%i|%s|%i|%s|%s|' % (
                            i, col, sam_value_counts_len, key, val))
                    flag = False
                else:
                    pp('||-|-|%s|%s|' % (key, val))
            if show_plots:
                plt.figure(i)
                ax = sam_value_counts.plot(kind='barh', figsize=figsize)
                _ = plt.title(col.upper())
                _ = plt.xlabel('counts')
        else:
            pp('|%i|%s|%i|||' % (i, col, sam_value_counts_len))
    return raw_markup_data

### Confusion Matrix

In [ ]:
from __future__ import division
import itertools
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')    

    
def confusion_maxtrix_stuff(y_test, y_pred, class_names):
    '''
    Example
    >>> confusion_maxtrix_stuff(y_test,
                                y_pred,
                                class_names=RAW_y.status_group.value_counts().keys()
                                ):
    '''
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    plt.figure(figsize=(8,8))
    plot_confusion_matrix(cnf_matrix, classes=class_names,
                          title='Confusion matrix, without normalization')
    # Plot normalized confusion matrix
    plt.figure(figsize=(8,8))
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized confusion matrix')
    plt.show()

## Import CSV Data

In [ ]:
RAW_X = pd.read_csv('traning_set_values.csv', index_col='id')
RAW_y = pd.read_csv('training_set_labels.csv', index_col='id')
RAW_TEST_X = pd.read_csv('test_set_values.csv', index_col='id')

In [ ]:
RAW_X.head().T

#### Observations:

* Most of the data seems categorical
* Following pairs looks closesly related
    * quantity & quantity_group
    * quality_group & water_quality
    * extraction_type, extraction_type_class & extraction_type_group
* Other
    * recorded_by, seems to hold only a single value
    * population & amount_tsh, values are for some given as zero

Lets check those group with 

In [ ]:
# %%asmarkdown
# show me the graphs
tmp = raw_markup_value_counts(dataframe=RAW_X, max_print_value_counts=10, show_plots=True, figsize=(9, 2))

## Pre Processing

** Num/Bool Tranformations **

* date_recorded --> Int
* longitude --> Float(less precision)
* latitude --> Float(less precision)
* public_meeting --> Bool
* permit --> Bool


Precision Description of Longititude and Latitude is available here at below link.
* http://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude

In [ ]:
from datetime import datetime
strptime = datetime.strptime

DATE_FORMAT = "%Y-%m-%d"
REFERENCE_DATE_POINT = strptime('2000-01-01', DATE_FORMAT)

# Reducing geo location precision to 11 meters
LONG_LAT_PRECISION = 0.001

def sam_datetime_to_number(x):
    return (strptime(str(x), DATE_FORMAT) - REFERENCE_DATE_POINT).days


# Transforming Date to Int.
if RAW_X.date_recorded.dtype == 'O':
    RAW_X.date_recorded = RAW_X.date_recorded.map(sam_datetime_to_number)
    RAW_TEST_X.date_recorded = RAW_TEST_X.date_recorded.map(sam_datetime_to_number)


# Filling Missing/OUTLIAR Values
_ = np.mean(RAW_X[u'latitude'][RAW_X.latitude < -1.0].values)
if not RAW_X.loc[RAW_X.latitude >= -1.0, u'latitude'].empty:
    RAW_X.loc[RAW_X.latitude >= -1.0, u'latitude'] = _
    RAW_TEST_X.loc[RAW_TEST_X.latitude >= -1.0, u'latitude'] = _


# Filling Missing/OUTLIAR Values
_ = np.mean(RAW_X[u'longitude'][RAW_X[u'longitude'] > 1.0].values)
if not RAW_X.loc[RAW_X[u'longitude'] <= 1.0, u'longitude'].empty:
    RAW_X.loc[RAW_X[u'longitude'] <= 1.0, u'longitude'] = _
    RAW_TEST_X.loc[RAW_TEST_X[u'longitude'] <= 1.0, u'longitude'] = _


# Reducing Precision of Lat.
if RAW_X.longitude.mean() < 50:
    RAW_X.longitude = RAW_X.longitude // LONG_LAT_PRECISION
    RAW_X.latitude = RAW_X.latitude // LONG_LAT_PRECISION
    RAW_TEST_X.longitude = RAW_TEST_X.longitude // LONG_LAT_PRECISION
    RAW_TEST_X.latitude = RAW_TEST_X.latitude // LONG_LAT_PRECISION


# Filling Missing/OUTLIAR Values
if RAW_X.public_meeting.dtype != 'bool':
    RAW_X.public_meeting = RAW_X.public_meeting == True
    RAW_TEST_X.public_meeting = RAW_TEST_X.public_meeting == True

if RAW_X.permit.dtype != 'bool':
    RAW_X.permit = RAW_X.permit == True
    RAW_TEST_X.permit = RAW_TEST_X.permit == True


# checking
if list(RAW_TEST_X.dtypes[RAW_TEST_X.dtypes != RAW_X.dtypes]):
    raise Exception('RAW_X.dtypes and RAW_TEST_X.dtypes are not in Sync')
else:
    print 'All in Good Shape'

** Text Data Tranformations **

We are going to basic clean action like, lower and upper case issue. Clearning of non ascii values.

In [ ]:
def text_transformation(name):
    if name:
        name = name.lower().strip()
        name = ''.join([i if 96 < ord(i) < 128 else ' ' for i in name])
        if 'and' in name:
            name = name.replace('and', ' ')
        if '/' in name:
            name = name.replace('/', ' ')
        while '  ' in name:
            name = name.replace('  ', ' ')
        return name.strip()
    return

In [ ]:
%%asmarkdown

print '''
|Column|Prev.|Current|
|------|-----|-------|'''
for col in RAW_X.dtypes[RAW_X.dtypes == object].index:
    aa = len(RAW_X[col].unique())
    RAW_X[col] = RAW_X[col].fillna('').apply(text_transformation)
    RAW_TEST_X[col] = RAW_TEST_X[col].fillna('').apply(text_transformation)
    bb = len(RAW_X[col].unique())
    if aa != bb:
        print '|%s|%i|%i|' % (col, aa, bb)

## Label Encoder

Label Encoder with DefaultDict for quick data transformation
http://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

In [ ]:
from collections import defaultdict
from sklearn import preprocessing

In [ ]:
d = defaultdict(preprocessing.LabelEncoder)

# Labels Fit
sam = pd.concat([RAW_X, RAW_TEST_X]).apply(lambda x: d[x.name].fit(x))

# Labels Transform - Training Data
X = RAW_X.apply(lambda x: d[x.name].transform(x))
TEST_X = RAW_TEST_X.apply(lambda x: d[x.name].transform(x))

le = preprocessing.LabelEncoder().fit(RAW_y[u'status_group'])
y = le.transform(RAW_y[u'status_group'])

In [ ]:
X.head(5)

In [ ]:
y[:5]

## Pickle Save

In [ ]:
pickle.dump(X, open('X.pkl', 'w'))
pickle.dump(TEST_X, open('TEST_X.pkl', 'w'))
pickle.dump(y, open('y.pkl', 'w'))
pickle.dump(d, open('d.pkl', 'w'))
pickle.dump(le, open('le.pkl', 'w'))

## Pickle Load

In [ ]:
X = pickle.load(open('X.pkl'))
y = pickle.load(open('y.pkl'))

In [ ]:
# Load this when you are about to do text transformation and submission

TEST_X = pickle.load(open('TEST_X.pkl'))
d = pickle.load(open('d.pkl'))
le = pickle.load(open('le.pkl'))

In [ ]:
X.shape, y.shape

In [ ]:
y[:5]

# Feature Selection


### Variance Threshold

To remove all features that are either one or zero (on or off) in more than 80% of the samples.

http://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance

http://stackoverflow.com/questions/29298973/removing-features-with-low-variance-scikit-learn/34850639#34850639

In [ ]:
from sklearn.feature_selection import VarianceThreshold

def get_low_variance_columns(dframe=None, columns=[],
                             skip_columns=[], threshold=0.0,
                             autoremove=False):
    """
    Wrapper for sklearn VarianceThreshold for use on pandas dataframes.
    """
    print("Finding low-variance features.")
    removed_features = []
    ranking_variance_thresholds = {}
    try:
        # get list of all the original df columns
        all_columns = dframe.columns

        # remove `skip_columns`
        remaining_columns = all_columns.drop(skip_columns)

        # get length of new index
        max_index = len(remaining_columns) - 1

        # get indices for `skip_columns`
        skipped_idx = [all_columns.get_loc(col)
                       for col
                       in skip_columns]

        # adjust insert location by the number of columns removed
        # (for non-zero insertion locations) to keep relative
        # locations intact
        for idx, item in enumerate(skipped_idx):
            if item > max_index:
                diff = item - max_index
                skipped_idx[idx] -= diff
            if item == max_index:
                diff = item - len(skip_columns)
                skipped_idx[idx] -= diff
            if idx == 0:
                skipped_idx[idx] = item

        # get values of `skip_columns`
        skipped_values = dframe.iloc[:, skipped_idx].values

        # get dataframe values
        X = dframe.loc[:, remaining_columns].values

        # instantiate VarianceThreshold object
        vt = VarianceThreshold(threshold=threshold)

        # fit vt to data
        vt.fit(X)

        # threshold ranking
        ranking_variance_thresholds = dict(zip(remaining_columns, vt.variances_))

        # get the indices of the features that are being kept
        feature_indices = vt.get_support(indices=True)

        # remove low-variance columns from index
        feature_names = [remaining_columns[idx]
                         for idx, _
                         in enumerate(remaining_columns)
                         if idx
                         in feature_indices]

        # get the columns to be removed
        removed_features = list(np.setdiff1d(remaining_columns,
                                             feature_names))
        print("Found {0} low-variance columns."
              .format(len(removed_features)))

        # remove the columns
        if autoremove:
            print("Removing low-variance features.")
            # remove the low-variance columns
            X_removed = vt.transform(X)

            print("Reassembling the dataframe (with low-variance "
                  "features removed).")
            # re-assemble the dataframe
            dframe = pd.DataFrame(data=X_removed,
                                  columns=feature_names)

            # add back the `skip_columns`
            for idx, index in enumerate(skipped_idx):
                dframe.insert(loc=index,
                              column=skip_columns[idx],
                              value=skipped_values[:, idx])
            print("Succesfully removed low-variance columns.")

        # do not remove columns
        else:
            print("No changes have been made to the dataframe.")

    except Exception as e:
        print(e)
        print("Could not remove low-variance features. Something "
              "went wrong.")
        return dframe, [], {}

    return dframe, removed_features, ranking_variance_thresholds

In [ ]:
X, removed_features, ranking_variance_thresholds = get_low_variance_columns(dframe=X,
                                                                            threshold=(0.8 * (1 - 0.8)),
                                                                            autoremove=True)

print '\nLow Variance Columns', removed_features
print 'Shape of X is', X.shape

In [ ]:
TEST_X.drop(removed_features, axis=1, inplace=True)

In [ ]:
print 'Shape of TEST_X is', TEST_X.shape

### Select K Best

* For regression: f_regression, mutual_info_regression
* For classification: chi2, f_classif, mutual_info_classif

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
SelectKBest?

In [ ]:
K_BEST_COLS = 25

fit = SelectKBest(score_func=chi2, k=K_BEST_COLS).fit(X, y)
cols_names = X.columns

np.set_printoptions(precision=2)

ranking_selectkbest = dict(zip(cols_names, fit.scores_))
selected_cols =  [_ for _ in cols_names[:K_BEST_COLS]]

features = pd.DataFrame(fit.transform(X))
features.columns = selected_cols

In [ ]:
% pprint
ranking_selectkbest

In [ ]:
print X.shape, features.shape, len(y)

In [ ]:
print 'Removed Columns:\n\t', '\n\t'.join([ _ for _ in X.columns if _ not in features.columns ])

print '\nSelected Columns:\n\t', '\n\t'.join(features.columns)

In [ ]:
print X.shape, features.shape

In [ ]:
X = pd.DataFrame(fit.transform(X))
X.columns = selected_cols

print

# TEST_X = pd.DataFrame(fit.transform(TEST_X))
# TEST_X.columns = selected_cols

### PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# feature extraction
pca = PCA(n_components=18)
fit = pca.fit(X)

In [ ]:
plt.figure(figsize=(12, 3))

_ = plt.scatter (range(len(fit.explained_variance_ratio_)), fit.explained_variance_ratio_.cumsum())

_ = plt.xlabel('cumsum of explained variance')

In [ ]:
X = pca.transform(X)
TEST_X = pca.transform(TEST_X)

# Unsupervised Learning

* Unsupervised Learning Exploration(Gaussian Process, Neural Nets)

### Gaussian Process

In [ ]:
from sklearn import mixture
import itertools
from scipy import linalg
import matplotlib as mpl

In [ ]:
mixture.GaussianMixture?

In [59]:
lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components,
                                      covariance_type=cv_type)
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

In [60]:
gmm

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=6, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [ ]:
bic = np.array(bic)
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                              'darkorange'])
clf = best_gmm
bars = []

In [53]:
cov.shape

(25,)

In [ ]:
# Plot the BIC scores
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

# Plot the winner
splot = plt.subplot(2, 1, 2)
Y_ = clf.predict(X)

for i, (mean, cov, color) in enumerate(zip(clf.means_, clf.covariances_,
                                           color_iter)):
    v, w = linalg.eigh(cov)
    if not np.any(Y_ == i):
        continue
    plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180. * angle / np.pi  # convert to degrees
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(.5)
    splot.add_artist(ell)

plt.xticks(())
plt.yticks(())
plt.title('Selected GMM: full model, 2 components')
plt.subplots_adjust(hspace=.35, bottom=.02)
plt.show()

In [ ]:
linalg.eigh?

### Neural Nets

## Supervised Learning

* Supervised Learning(GBT Trees, Nearest Neighbours, RF, One-vs-One)

### Test-Train Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb

In [ ]:
print X.shape
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print features.shape
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.25, random_state=42, stratify=y)

### GBT Trees

### Nearest Neighbours

### Random Forest

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=150, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1)
# class_weight="balanced_subsample"/"balanced"
# criterion="gini"/"entropy"

rf_clf = rf_clf.fit(X_train, y_train)
# pred = rf_clf.predict_proba(X_test)
rf_clf.score(X_test, y_test)

# (n_estimators=100, class_weight="balanced_subsample", n_jobs=-1) 0.80782828282828278
# (n_estimators=100, class_weight="balanced_subsample", n_jobs=-1) 0.81186868686868685
# (n_estimators=150, class_weight="balanced_subsample", n_jobs=-1) 0.8113636363636364

# (n_estimators=150, criterion='gini', class_weight="balanced_subsample", n_jobs=-1) 0.81018518518518523
# (n_estimators=150, criterion='gini', class_weight="balanced", n_jobs=-1) 0.80858585858585863
# (n_estimators=150, criterion='gini', n_jobs=-1) 0.80942760942760938
# (n_estimators=150, criterion='entropy', n_jobs=-1) 0.81060606060606055

# (n_estimators=10, criterion='gini', class_weight="balanced_subsample", n_jobs=-1) 0.79781144781144786
# (n_estimators=10, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1) 0.80185185185185182
# (n_estimators=150, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1) 0.8113636363636364
# (n_estimators=150, criterion='gini', class_weight="balanced", n_jobs=-1) 0.80984848484848482

# (n_estimators=150, criterion='gini', class_weight="balanced", n_jobs=-1) 0.81259259259259264
# (n_estimators=150, criterion='gini', class_weight="balanced_subsample", n_jobs=-1) 0.81259259259259264
# (n_estimators=150, criterion='entropy', class_weight="balanced_subsample", n_jobs=-1) 0.81252525252525254

### One-vs-One

## Parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
RandomForestClassifier?

In [ ]:
parameters = {
    n_estimators: [10, 50, 100],
    class_weight: 'balanced_subsample balanced'.split(),
    criterion: 'gini entropy'.split(),
#     n_jobs: [-1]
}
GS_CV = GridSearchCV(estimator=RandomForestClassifier(), param_grid=parameters, n_jobs=-1)

GS_CV.fit(features, y)

In [ ]:
sorted(clf.cv_results_.keys())

In [ ]:
len(clf.feature_importances_)

In [ ]:
plt.figure(figsize=(12, 3))

tmp = clf.feature_importances_

# making importance relative
a, b = min(tmp), max(tmp)
cols_imp = (tmp - a) /b
_ = plt.scatter(range(30), cols_imp)
_ = plt.plot((0, 29), (0.15,0.15), '-r')
_ = plt.xlabel('Columns')
_ = plt.ylabel('Relative Col Importance')

In [ ]:
print map(lambda x: len(x), [X_test, y_test])

clf.score(X_test, y_test) # 0.81097643097643102

### XGBOOST

## Submission

In [ ]:
test_ids = RAW_TEST_X.index

predictions = clf.predict(TEST_X)

print (predictions.shape)

predictions_labels = le.inverse_transform(predictions)

# sub = pd.DataFrame(predictions, columns=list(le.classes_))
sub = pd.DataFrame(predictions_labels, columns=['status_group'])
sub.head()

sub.insert(loc=0, column='id', value=test_ids)
sub.reset_index()
sub.to_csv('submit.csv', index = False)
sub.head()